In [2]:
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
import numpy as np
import seaborn as sns
import plotly.express as px
import matplotlib.ticker as mtick
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [3]:
with pd.io.stata.StataReader("../dta files/2012.dta", convert_categoricals=True) as rdr:
    cols = [c for c in rdr.variable_labels().keys() if c not in {"ISCO88", "SPISCO88"}]
    df_2012 = rdr.read(columns=cols)

with pd.io.stata.StataReader("../dta files/2002.dta", convert_categoricals=True) as rdr:
    cols = [c for c in rdr.variable_labels().keys() if c not in ["v241","v247"]]
    df_2002 = rdr.read(columns=cols)

df_2022 = pd.read_stata("../dta files/2022.dta")


In [316]:
df_2002_clean = pd.read_csv("../data/final_csv/2002_final.csv")
df_2012_clean = pd.read_csv("../data/final_csv/2012_final.csv")
df_2022_clean = pd.read_csv("../data/final_csv/2022_final.csv")

In [317]:
dfs = [df_2002_clean, df_2012_clean, df_2022_clean]
cleaned_vars = [
    "urban_rural",
    "spouse_work_status",
    "sex",
    "code_higher_income",
    "code_income_control",
    "hh_wrk_hrs",
    "work_status",
    "marital",
    "wrk_hrs",
    "educ_4_label",
    "age",
]
for df in dfs:
    for var in cleaned_vars:
        print(var)
        print(df[var].unique())

urban_rural
['Urban' 'Town' 'Rural' nan]
spouse_work_status
['Retired' 'Paid work' 'Domestic work' 'DK/No answer' 'Other' 'Unemployed'
 'Education' 'Help family member' 'Sick/Disabled']
sex
['Female' 'Male' nan]
code_higher_income
['Respondent has higher income' nan 'Same income'
 'Partner has higher income' 'Partner has no income'
 'Respondent has no income']
code_income_control
[nan 'Financial control is shared'
 'Financial control is with the respondent'
 'Financial control is separate' 'Financial control is with the partner']
hh_wrk_hrs
[30.  5. 16. 28. 12. 10. 20. 26.  8. 35.  3. 23. 50. 14.  7. 18.  4. nan
  6.  9. 15. 40.  2. 25. 21. 39. 60. 24. 11. 56. 37. 22. 70. 17. 45. 65.
 38. 31. 27. 32.  0. 36. 42. 48. 49. 46. 52. 13. 55.  1. 29. 19. 90. 84.
 80. 72. 34. 51. 54. 95. 85. 47. 58. 78. 33. 44. 75. 43. 82. 63. 86. 59.
 64. 41. 68. 62. 74. 87. 77. 94. 67. 66. 57. 71. 88. 76. 91.]
work_status
['Paid work' 'Unemployed' 'Retired' 'Domestic work' 'Other' 'Education'
 'DK/No answer'

In [318]:
def normalize_column(series):
    """
    Normalize a numeric series to 0-1 range using min-max scaling.
    """
    return (series - series.min()) / (series.max() - series.min())

# Apply to eg_score columns
df_2002_clean['eg_score_norm'] = normalize_column(df_2002_clean['eg_score'])
df_2012_clean['eg_score_norm'] = normalize_column(df_2012_clean['eg_score'])
df_2022_clean['eg_score_norm'] = normalize_column(df_2022_clean['eg_score'])

In [319]:
df_mapping = pd.read_csv("../common_question_mapping.csv")

In [320]:
for index, row in df_mapping.iterrows():
    if row["COMMON_VAR"] in df_2022_clean.columns:
        print(row["COMMON_VAR"])

urban_rural
spouse_work_status
sex
code_higher_income
code_income_control
hh_wrk_hrs
work_status
marital
wrk_hrs
educ_4_label
age


In [321]:
import re

for index, row in df_mapping.iterrows():
    # Fix var_02: should start with lowercase "v"
    if row["COMMON_VAR"] not in df_2022_clean.columns:
        if pd.notna(row["var_02"]) and isinstance(row["var_02"], str):
            if re.match(r'^V\d', row["var_02"]):
                df_mapping.at[index, "var_02"] = 'v' + row["var_02"][1:]
        
        # Fix var_12: should start with uppercase "V"
        if pd.notna(row["var_12"]) and isinstance(row["var_12"], str):
            if re.match(r'^v\d', row["var_12"]):
                df_mapping.at[index, "var_12"] = 'V' + row["var_12"][1:]
        
        # Fix var_22: should start with lowercase "v"
        if pd.notna(row["var_22"]) and isinstance(row["var_22"], str):
            if re.match(r'^V\d', row["var_22"]):
                df_mapping.at[index, "var_22"] = 'v' + row["var_22"][1:]

print("✓ Variable names standardized:")
print("  - var_02 (2002): lowercase 'v'")
print("  - var_12 (2012): uppercase 'V'")
print("  - var_22 (2022): lowercase 'v'")

✓ Variable names standardized:
  - var_02 (2002): lowercase 'v'
  - var_12 (2012): uppercase 'V'
  - var_22 (2022): lowercase 'v'


In [322]:
# # Add common variables to clean dataframes from raw dataframes
# # Ensure all COMMON_VAR columns exist in all dataframes for consistency
# for index, row in df_mapping.iterrows():
#     common_var = row["COMMON_VAR"]
#     if common_var in df_2002_clean.columns:
#         print(common_var)
#     if common_var not in df_2002_clean.columns:
#         # 2002
#         if pd.notna(row["var_02"]) and row["var_02"] in df_2002.columns:
#             df_2002_clean[common_var] = df_2002[row["var_02"]].values
#         else:
#             # Create column with NaN if variable doesn't exist for this year
#             if common_var not in df_2002_clean.columns:
#                 df_2002_clean[common_var] = np.nan
        
#         # 2012
#         if pd.notna(row["var_12"]) and row["var_12"] in df_2012_clean.columns:
#             df_2012_clean[common_var] = df_2012[row["var_12"]].values
#         else:
#             # Create column with NaN if variable doesn't exist for this year
#             if common_var not in df_2012_clean.columns:
#                 df_2012_clean[common_var] = np.nan
        
#         # 2022
#         if pd.notna(row["var_22"]) and row["var_22"] in df_2022.columns:
#             df_2022_clean[common_var] = df_2022[row["var_22"]].values
#         else:
#             # Create column with NaN if variable doesn't exist for this year
#             if common_var not in df_2022_clean.columns:
#                 df_2022_clean[common_var] = np.nan



import numpy as np
import pandas as pd

for _, row in df_mapping.iterrows():
    common_var = row["COMMON_VAR"]

    # ---- 2002 ----
    if common_var not in df_2002_clean.columns:
        v = row.get("var_02", np.nan)
        if pd.notna(v) and v in df_2002.columns:
            df_2002_clean[common_var] = df_2002[v].reindex(df_2002_clean.index)
        else:
            df_2002_clean[common_var] = np.nan

    # ---- 2012 ----
    if common_var not in df_2012_clean.columns:
        v = row.get("var_12", np.nan)
        if pd.notna(v) and v in df_2012.columns:
            df_2012_clean[common_var] = df_2012[v].reindex(df_2012_clean.index)
        else:
            df_2012_clean[common_var] = np.nan

    # ---- 2022 ----
    if common_var not in df_2022_clean.columns:
        v = row.get("var_22", np.nan)
        if pd.notna(v) and v in df_2022.columns:
            df_2022_clean[common_var] = df_2022[v].reindex(df_2022_clean.index)
        else:
            df_2022_clean[common_var] = np.nan


print(f"✓ Added {len(df_mapping)} common variables to all clean dataframes")
print(f"  2002: {df_2002_clean.shape}")
print(f"  2012: {df_2012_clean.shape}")
print(f"  2022: {df_2022_clean.shape}")

✓ Added 47 common variables to all clean dataframes
  2002: (46638, 69)
  2012: (61754, 69)
  2022: (45762, 69)


In [323]:
dfs = [df_2002_clean, df_2012_clean, df_2022_clean]
cleaned_vars = [
    "urban_rural",
    "spouse_work_status",
    "sex",
    "code_higher_income",
    "code_income_control",
    "hh_wrk_hrs",
    "work_status",
    "marital",
    "wrk_hrs",
    "educ_4_label",
    "age",
]
for df in dfs:
    for var in cleaned_vars:
        print(var)
        print(df[var].unique())

urban_rural
['Urban' 'Town' 'Rural' nan]
spouse_work_status
['Retired' 'Paid work' 'Domestic work' 'DK/No answer' 'Other' 'Unemployed'
 'Education' 'Help family member' 'Sick/Disabled']
sex
['Female' 'Male' nan]
code_higher_income
['Respondent has higher income' nan 'Same income'
 'Partner has higher income' 'Partner has no income'
 'Respondent has no income']
code_income_control
[nan 'Financial control is shared'
 'Financial control is with the respondent'
 'Financial control is separate' 'Financial control is with the partner']
hh_wrk_hrs
[30.  5. 16. 28. 12. 10. 20. 26.  8. 35.  3. 23. 50. 14.  7. 18.  4. nan
  6.  9. 15. 40.  2. 25. 21. 39. 60. 24. 11. 56. 37. 22. 70. 17. 45. 65.
 38. 31. 27. 32.  0. 36. 42. 48. 49. 46. 52. 13. 55.  1. 29. 19. 90. 84.
 80. 72. 34. 51. 54. 95. 85. 47. 58. 78. 33. 44. 75. 43. 82. 63. 86. 59.
 64. 41. 68. 62. 74. 87. 77. 94. 67. 66. 57. 71. 88. 76. 91.]
work_status
['Paid work' 'Unemployed' 'Retired' 'Domestic work' 'Other' 'Education'
 'DK/No answer'

In [326]:
df_2022_clean.shape

(45762, 69)

In [327]:
df_2002_clean.to_csv("../new_clean_csv/2002_clean.csv", index=False)
df_2012_clean.to_csv("../new_clean_csv/2012_clean.csv", index=False)
df_2022_clean.to_csv("../new_clean_csv/2022_clean.csv", index=False)

print("✓ Saved clean dataframes to ../clean_csv/")

✓ Saved clean dataframes to ../clean_csv/


In [328]:
# Read the saved clean CSVs

df_2002_clean = pd.read_csv("../new_clean_csv/2002_clean.csv")
print(f"2022: {df_2022_clean.shape}")

df_2012_clean = pd.read_csv("../new_clean_csv/2012_clean.csv")
print(f"2012: {df_2012_clean.shape}")

df_2022_clean = pd.read_csv("../new_clean_csv/2022_clean.csv")
print(f"2002: {df_2002_clean.shape}")


2022: (45762, 69)
2012: (61754, 69)
2002: (46638, 69)


In [329]:
df_2002_clean

,v4_egal,v5_egal,v6_egal,v7_egal,v8_egal,v11_egal,v4_egal_z,v5_egal_z,v6_egal_z,v7_egal_z,...,LIFE_HAP,DIFF_CONC_WORK,HH_TIRED,HH_FAM,WORK_TIRED,HH_WEEKEND,COHAB,HOMPOP,COUNTRY,CASEID
0,2.0,2.0,2.0,3.0,3.0,3.0,-1.357029,-0.614036,-0.676980,0.125346,...,Neither happy nor unhappy,Several times a month,Never,NaN,Several times a month,Mostly me,NaN,2 persons,Australia (AU),1000001
1,5.0,5.0,5.0,5.0,1.0,5.0,1.105198,1.788120,1.680397,1.741713,...,Very happy,NaN,NaN,NaN,NaN,We decide together,NaN,2 persons,Australia (AU),1000002
2,4.0,3.0,2.0,3.0,2.0,4.0,0.284456,0.186683,-0.676980,0.125346,...,Fairly happy,NaN,NaN,NaN,NaN,Mostly me,NaN,2 persons,Australia (AU),1000003
3,2.0,2.0,1.0,4.0,4.0,4.0,-1.357029,-0.614036,-1.462772,0.933529,...,Very happy,Never,Never,NaN,Never,We decide together,NaN,2 persons,Australia (AU),1000004
4,3.0,2.0,2.0,3.0,2.0,3.0,-0.536287,-0.614036,-0.676980,0.125346,...,Very happy,Once or twice a year,Never,NaN,Once or twice a year,We decide together,NaN,2 persons,Australia (AU),1000005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46633,4.0,4.0,4.0,2.0,2.0,4.0,0.284456,0.987401,0.894605,-0.682838,...,Very happy,Never,Never,NaN,Several times a month,NaN,No,3 persons,Taiwan (TW),39093339
46634,4.0,2.0,NaN,1.0,2.0,1.0,0.284456,-0.614036,NaN,-1.491022,...,Completely happy,NaN,NaN,NaN,NaN,NaN,No,1 person,Taiwan (TW),39093340
46635,4.0,2.0,4.0,2.0,2.0,3.0,0.284456,-0.614036,0.894605,-0.682838,...,Fairly happy,Once or twice a year,Never,NaN,Once or twice a year,Mostly me,NaN,5 persons,Taiwan (TW),39093341
46636,4.0,4.0,4.0,4.0,2.0,4.0,0.284456,0.987401,0.894605,0.933529,...,Fairly happy,Never,Never,NaN,Several times a month,NaN,No,7 persons,Taiwan (TW),39093342


In [330]:
cleaned_vars = [
    "urban_rural",
    "spouse_work_status",
    "sex",
    "code_higher_income",
    "code_income_control",
    "hh_wrk_hrs",
    "work_status",
    "marital",
    "wrk_hrs",
    "educ_4_label",
    "age",
]


In [331]:
dfs = [df_2002_clean, df_2012_clean, df_2022_clean]
year = [2002, 2012, 2022]
count = 0
for col_name in df_mapping["COMMON_VAR"]:
    if col_name not in cleaned_vars:
        count+=1
        print("="*20,col_name, "="*20)
        for df, yr in zip(dfs, year):
            print("="*20, f"df_{yr}_clean", "="*20)
            print(df[col_name].unique())

==================== TOPBOT ====================
==================== df_2002_clean ====================
['5.0' '8.0' '6.0' '4.0' '7.0' nan '3.0' 'Lowest' '2.0' '9.0' 'Highest']
==================== df_2012_clean ====================
['05' '07' '08' '03' '06' '04' 'Lowest, Bottom, 01' "Don't know" '02'
 'Highest, Top, 10' '09' 'No answer' 'Refused' 'Not available: GB,US']
==================== df_2022_clean ====================
['4. 04' '1. Lowest, Bottom, 01' '8. 08' '5. 05' '7. 07' '6. 06'
 "-8. Don't know" '10. Highest, Top, 10' '9. 09' '2. 02' '3. 03'
 '-9. No answer' '-1. CZ: Not available']
==================== SPWRKHRS ====================
==================== df_2002_clean ====================
[nan '40 hours' '37.0' '10 hours' '45.0' '20 hours' '12.0' '55.0' '38.0'
 '30 hours' '42.0' '48.0' '50 hours' '58.0' '44.0' '25.0' '19.0' '60.0'
 '28.0' '35.0' '39.0' '8.0' '15 hours' '22.0' '43.0' '3.0' '16.0' '72.0'
 '80.0' '24.0' '70.0' '65.0' '96 hours and more' '41.0' '11.0' '18.0'
 '

In [332]:
count

36

In [333]:
import re

# ==================== CLEANING FUNCTIONS FOR ALL COMMON VARIABLES ====================

# 1. LIKERT SCALE (5-point: Strongly Agree to Strongly Disagree)
def clean_likert_5(val):
    """Standardize 5-point Likert scale responses."""
    if pd.isna(val):
        return None
    
    val_str = str(val).lower()
    
    # Handle missing values
    if any(x in val_str for x in ['no answer', "don't know", 'refused', 'nap', "can't choose", 'not available']):
        return None
    
    # Map to standard categories
    if 'strongly agree' in val_str or val_str.startswith('1.'):
        return "Strongly agree"
    if ('agree' in val_str and 'disagree' not in val_str and 'neither' not in val_str) or val_str.startswith('2.'):
        return "Agree"
    if 'neither' in val_str or val_str.startswith('3.'):
        return "Neither agree nor disagree"
    if ('disagree' in val_str and 'strongly' not in val_str) or val_str.startswith('4.'):
        return "Disagree"
    if 'strongly disagree' in val_str or val_str.startswith('5.'):
        return "Strongly disagree"
    
    return None

# 2. INCOME DECILE (TOPBOT)
def clean_income_decile(val):
    """Extract numeric decile from income scale (1-10)."""
    if pd.isna(val):
        return None
    
    val_str = str(val).lower()
    
    # Handle missing values
    if any(x in val_str for x in ["don't know", 'no answer', 'refused', 'not available']):
        return None
    
    # Handle text labels
    if 'lowest' in val_str or 'bottom' in val_str or '01' in val_str or val_str.startswith('1.'):
        return 1
    if 'highest' in val_str or 'top' in val_str or '10' in val_str:
        return 10
    
    # Extract numeric value
    match = re.search(r'(\d+)', val_str)
    if match:
        num = int(match.group(1))
        if 1 <= num <= 10:
            return num
    
    return None

# 3. WORK HOURS (SPWRKHRS, SP_HH_FAM, SP_HH, HH_FAM)
def clean_work_hours(val):
    """Extract numeric work hours (0-95+)."""
    if pd.isna(val):
        return None
    
    val_str = str(val).lower()
    
    # Handle missing/NAP
    if any(x in val_str for x in ['no answer', "don't know", 'refused', 'nap', 'not available']):
        return None
    
    # Handle "none"
    if 'none' in val_str or 'no hour' in val_str:
        return 0
    
    # Handle "one hour"
    if 'one hour' in val_str or val_str == '1. 1 hour':
        return 1
    
    # Handle 95+ 
    if '95' in val_str or '96' in val_str:
        return 95
    
    # Extract numeric value (handle floats too)
    match = re.search(r'(\d+\.?\d*)', val_str)
    if match:
        num = float(match.group(1))
        return int(num) if num <= 95 else 95
    
    return None

# 4. WORK PREFERENCE (WWYKS, WWYKUS)
def clean_work_preference(val):
    """Standardize work preference categories."""
    if pd.isna(val):
        return None
    
    val_str = str(val).lower()
    
    # Handle missing
    if any(x in val_str for x in ['no answer', "don't know", "can't choose"]):
        return None
    
    # Women should decide
    if 'women should decide' in val_str or 'women shld decide' in val_str or val_str.startswith('4.'):
        return "Women should decide"
    
    # Work full-time
    if 'full-time' in val_str or 'full time' in val_str or val_str.startswith('1.'):
        return "Work full-time"
    
    # Work part-time
    if 'part-time' in val_str or 'part time' in val_str or val_str.startswith('2.'):
        return "Work part-time"
    
    # Stay at home
    if 'stay at home' in val_str or val_str.startswith('3.'):
        return "Stay at home"
    
    return None

# 5. EDUCATION LEVEL (SP_DEGREE)
def clean_education(val):
    """Standardize education levels."""
    if pd.isna(val):
        return None
    
    val_str = str(val).lower()
    
    # Handle missing/NAP
    if any(x in val_str for x in ['no answer', "don't know", 'refused', 'nap', 'not available', 'not classifiable']):
        return None
    
    # Map to standardized categories
    if 'no formal' in val_str or 'no (formal) education' in val_str or val_str.startswith('0.'):
        return "No formal education"
    
    if 'primary' in val_str and 'incomplete' not in val_str or val_str.startswith('1.'):
        return "Primary"
    
    if 'lower secondary' in val_str or val_str.startswith('2.'):
        return "Lower secondary"
    
    if 'upper secondary' in val_str or 'higher secondary' in val_str or val_str.startswith('3.'):
        return "Upper secondary"
    
    if 'post secondary' in val_str or val_str.startswith('4.'):
        return "Post secondary"
    
    if ('short-cycle tertiary' in val_str or 'lower level tertiary' in val_str or 
        'above lowest qualification' in val_str or val_str.startswith('5.') or val_str.startswith('6.')):
        return "Tertiary (Bachelor level)"
    
    if ('upper level tertiary' in val_str or 'university degree' in val_str or 
        'above higher secondary' in val_str or val_str.startswith('7.')):
        return "Tertiary (Master level)"
    
    if 'phd' in val_str or 'post tertiary' in val_str or val_str.startswith('8.'):
        return "Tertiary (Doctoral level)"
    
    if 'lowest formal qualification' in val_str:
        return "Primary"
    
    return None

# 6. PARENT SUITABILITY (MOMORFAF)
def clean_parent_suit(val):
    """Standardize parent suitability scale."""
    if pd.isna(val):
        return None
    
    val_str = str(val).lower()
    
    # Handle missing
    if any(x in val_str for x in ['no answer', "don't know", "can't choose"]):
        return None
    
    # 2002 uses Likert, 2022 uses specific scale
    if 'strongly agree' in val_str or 'mothers and fathers are equally' in val_str or val_str.startswith('3.'):
        return "Equally suited"
    
    if ('agree' in val_str and 'strongly' not in val_str and 'neither' not in val_str and 'disagree' not in val_str) or \
       'mothers are somewhat better' in val_str or val_str.startswith('2.'):
        return "Mothers somewhat better"
    
    if 'mothers are much better' in val_str or val_str.startswith('1.'):
        return "Mothers much better"
    
    if 'fathers are somewhat better' in val_str or val_str.startswith('4.'):
        return "Fathers somewhat better"
    
    if 'strongly disagree' in val_str or 'fathers are much better' in val_str or val_str.startswith('5.'):
        return "Fathers much better"
    
    if 'neither' in val_str or 'disagree' in val_str:
        return "Equally suited"
    
    return None

# 7. HOUSEHOLD COUNTS (HHTODD, HHCHILDR, HHADULT, HOMPOP)
def clean_household_count(val):
    """Extract numeric count from household composition variables."""
    if pd.isna(val):
        return None
    
    val_str = str(val).lower()
    
    # Handle missing/NAP
    if any(x in val_str for x in ['no answer', 'refused', 'nap', 'not available']):
        return None
    
    # Handle "no children/adults/toddlers/persons"
    if any(x in val_str for x in ['no children', 'no toddlers', 'no adults', 'no persons']):
        return 0
    
    # Handle "one"
    if 'one child' in val_str or 'one toddler' in val_str or 'one adult' in val_str or 'one person' in val_str:
        return 1
    
    # Extract numeric value
    match = re.search(r'(\d+)', val_str)
    if match:
        return int(match.group(1))
    
    return None

# 8. FREQUENCY SCALE (FAM_DIF, DIFF_CONC_WORK, HH_TIRED, WORK_TIRED)
FREQUENCY_ORDER = [
    "Several times a week",
    "Several times a month",
    "Once or twice",
    "Never",
    "NAP"
]

def clean_frequency(val):
    """Standardize frequency scale."""
    if pd.isna(val):
        return "NAP"
    
    val_str = str(val).lower()
    
    # Handle missing/NAP
    if any(x in val_str for x in ['no answer', "don't know", 'refused', "doesn't apply", 'nap', 'not available', "can't choose"]):
        return "NAP"
    
    if 'several times a week' in val_str or val_str.startswith('1.'):
        return "Several times a week"
    
    if 'several times a month' in val_str or val_str.startswith('2.'):
        return "Several times a month"
    
    if 'once or twice' in val_str or val_str.startswith('3.'):
        return "Once or twice"
    
    if 'never' in val_str or val_str.startswith('4.'):
        return "Never"
    
    return "NAP"

# 9. TASK FAIRNESS SHARE (SHARE_HH)
FAIRNESS_SHARE_ORDER = [
    "Much more than fair share",
    "Bit more than fair share",
    "Fair share",
    "Bit less than fair share",
    "Much less than fair share",
    "NAP"
]

def clean_fairness_share(val):
    """Standardize task fairness perception."""
    if pd.isna(val):
        return "NAP"
    
    val_str = str(val).lower()
    
    # Handle missing/NAP
    if any(x in val_str for x in ['no answer', "don't know", 'refused', 'nap', "can't choose"]):
        return "NAP"
    
    if 'much more' in val_str or val_str.startswith('1.'):
        return "Much more than fair share"
    
    if ('bit more' in val_str or 'a bit more' in val_str) or val_str.startswith('2.'):
        return "Bit more than fair share"
    
    if 'roughly my fair share' in val_str or ('fair share' in val_str and 'more' not in val_str and 'less' not in val_str) or val_str.startswith('3.'):
        return "Fair share"
    
    if ('bit less' in val_str or 'a bit less' in val_str) or val_str.startswith('4.'):
        return "Bit less than fair share"
    
    if 'much less' in val_str or val_str.startswith('5.'):
        return "Much less than fair share"
    
    return "NAP"

# 10. TASK DIVISION (DIV_HH_* variables)
TASK_DIV_ORDER = [
    "Always respondent",
    "Usually respondent",
    "About equal",
    "Usually partner",
    "Always partner",
    "Third person",
    "NAP"
]

def clean_task_div(val):
    """Standardize task division categories."""
    if pd.isna(val):
        return "NAP"
    
    val_str = str(val).lower()
    
    # Handle missing/NAP
    if any(x in val_str for x in ['no answer', "don't know", 'refused', 'nap', "can't choose"]):
        return "NAP"
    
    # Third person first
    if 'third person' in val_str or val_str.startswith('6.'):
        return "Third person"
    
    # Always respondent/me
    if ('always me' in val_str or 'always respondent' in val_str) or val_str.startswith('1.'):
        return "Always respondent"
    
    # Usually respondent/me
    if ('usually me' in val_str or 'usually respondent' in val_str) or val_str.startswith('2.'):
        return "Usually respondent"
    
    # About equal
    if 'about equal' in val_str or 'both together' in val_str or 'both equally' in val_str or val_str.startswith('3.'):
        return "About equal"
    
    # Usually partner
    if ('usually' in val_str and ('spouse' in val_str or 'partner' in val_str)) or val_str.startswith('4.'):
        return "Usually partner"
    
    # Always partner
    if ('always' in val_str and ('spouse' in val_str or 'partner' in val_str)) or val_str.startswith('5.'):
        return "Always partner"
    
    return "NAP"

# 11. HAPPINESS SCALE (LIFE_HAP)
HAPPINESS_ORDER = [
    "Completely happy",
    "Very happy",
    "Fairly happy",
    "Neither happy nor unhappy",
    "Fairly unhappy",
    "Very unhappy",
    "Completely unhappy"
]

def clean_happiness(val):
    """Standardize happiness scale."""
    if pd.isna(val):
        return None
    
    val_str = str(val).lower()
    
    # Handle missing
    if any(x in val_str for x in ['no answer', "don't know", "can't choose"]):
        return None
    
    if 'completely happy' in val_str or val_str.startswith('1.'):
        return "Completely happy"
    if 'very happy' in val_str or val_str.startswith('2.'):
        return "Very happy"
    if 'fairly happy' in val_str or val_str.startswith('3.'):
        return "Fairly happy"
    if 'neither' in val_str or val_str.startswith('4.'):
        return "Neither happy nor unhappy"
    if 'fairly unhappy' in val_str or val_str.startswith('5.'):
        return "Fairly unhappy"
    if 'very unhappy' in val_str or val_str.startswith('6.'):
        return "Very unhappy"
    if 'completely unhappy' in val_str or val_str.startswith('7.'):
        return "Completely unhappy"
    
    return None

# 12. WEEKEND DECISION (HH_WEEKEND)
def clean_weekend_decision(val):
    """Standardize weekend activity decision variable."""
    if pd.isna(val):
        return "NAP"
    
    val_str = str(val).lower()
    
    # Handle missing/NAP
    if any(x in val_str for x in ['no answer', "don't know", 'refused', 'nap', "can't choose"]):
        return "NAP"
    
    # Map to task division categories for consistency
    if 'always me' in val_str or 'mostly me' in val_str or val_str.startswith('1.'):
        return "Always respondent"
    
    if 'usually me' in val_str or val_str.startswith('2.'):
        return "Usually respondent"
    
    if 'we decide together' in val_str or 'about equal' in val_str or 'both together' in val_str or val_str.startswith('3.'):
        return "About equal"
    
    if ('usually' in val_str and ('spouse' in val_str or 'partner' in val_str)) or val_str.startswith('4.'):
        return "Usually partner"
    
    if ('always' in val_str and ('spouse' in val_str or 'partner' in val_str)) or 'mostly my spouse' in val_str or val_str.startswith('5.'):
        return "Always partner"
    
    if 'third person' in val_str or 'someone else' in val_str or val_str.startswith('6.'):
        return "Third person"
    
    if 'sometimes' in val_str:
        return "About equal"
    
    return "NAP"

# 13. COHABITATION (COHAB)
def clean_cohab(val):
    """Standardize cohabitation status."""
    if pd.isna(val):
        return None
    
    val_str = str(val).lower()
    
    # Handle missing
    if any(x in val_str for x in ['no answer', 'refused', 'not available']):
        return None
    
    if 'yes' in val_str and 'same household' in val_str or val_str.startswith('1.'):
        return "Partner, same household"
    
    if 'yes' in val_str and "don't live" in val_str or val_str.startswith('2.'):
        return "Partner, different household"
    
    if 'no partner' in val_str or val_str.startswith('3.'):
        return "No partner"
    
    # Legacy coding
    if val_str == 'yes':
        return "Partner, same household"
    if val_str == 'no':
        return "No partner"
    
    return None

# 14. COUNTRY CODE (C_ALPHAN)
def clean_country(val):
    """Extract standardized country code."""
    if pd.isna(val):
        return None
    
    val_str = str(val).upper()
    
    # Extract 2-letter ISO code at start or after dash
    match = re.match(r'(\d+\.\s*)?([A-Z]{2})', val_str)
    if match:
        return match.group(2)
    
    return val_str[:2] if len(val_str) >= 2 else None

print("✓ Defined cleaning functions for all common variables")
print("\nFunction categories:")
print("  - Likert scales (5-point)")
print("  - Income deciles")
print("  - Work hours")
print("  - Work preferences")
print("  - Education levels")
print("  - Parent suitability")
print("  - Household counts")
print("  - Frequency scales")
print("  - Task fairness")
print("  - Task division")
print("  - Happiness scale")
print("  - Weekend decisions")
print("  - Cohabitation status")
print("  - Country codes")

✓ Defined cleaning functions for all common variables

Function categories:
  - Likert scales (5-point)
  - Income deciles
  - Work hours
  - Work preferences
  - Education levels
  - Parent suitability
  - Household counts
  - Frequency scales
  - Task fairness
  - Task division
  - Happiness scale
  - Weekend decisions
  - Cohabitation status
  - Country codes


In [334]:
# ==================== APPLY CLEANING TO ALL VARIABLES ====================

# Variable mapping: column_name -> cleaning_function
cleaning_map = {
    # Income decile
    'TOPBOT': clean_income_decile,
    
    # Work hours
    'SPWRKHRS': clean_work_hours,
    'SP_HH_FAM': clean_work_hours,
    'SP_HH': clean_work_hours,
    'HH_FAM': clean_work_hours,
    
    # Work preferences
    'WWYKS': clean_work_preference,
    'WWYKUS': clean_work_preference,
    
    # Education
    'SP_DEGREE': clean_education,
    
    # Parent suitability
    'MOMORFAF': clean_parent_suit,
    
    # Household counts
    'HHTODD': clean_household_count,
    'HHCHILDR': clean_household_count,
    'HHADULT': clean_household_count,
    'HOMPOP': clean_household_count,
    
    # Likert scales
    'LIVWOMAR': clean_likert_5,
    'MEWH': clean_likert_5,
    'HW_FULFIL': clean_likert_5,
    'WO_WANT': clean_likert_5,
    'WW_FAM_SUFFER': clean_likert_5,
    'WW_CHILD_SUFFER': clean_likert_5,
    'WW_WARM': clean_likert_5,
    
    # Frequency scales
    'FAM_DIF': clean_frequency,
    'DIFF_CONC_WORK': clean_frequency,
    'HH_TIRED': clean_frequency,
    'WORK_TIRED': clean_frequency,
    
    # Task fairness share
    'SHARE_HH': clean_fairness_share,
    
    # Task division
    'DIV_HH_COOK': clean_task_div,
    'DIV_HH_CLEAN': clean_task_div,
    'DIV_HH_GROC': clean_task_div,
    'DIV_HH_CARE': clean_task_div,
    'DIV_HH_LAUND': clean_task_div,
    'HH_WEEKEND': clean_weekend_decision,
    
    # Happiness
    'LIFE_HAP': clean_happiness,
    
    # Cohabitation
    'COHAB': clean_cohab,
    
    # Country
    'C_ALPHAN': clean_country,
}

# Apply cleaning functions to all dataframes
for col_name, clean_func in cleaning_map.items():
    for df in [df_2002_clean, df_2012_clean, df_2022_clean]:
        if col_name in df.columns:
            df[col_name] = df[col_name].apply(clean_func)

print(f"✓ Applied cleaning to {len(cleaning_map)} variables across all years")

# Convert categorical variables to ordered categories where appropriate
categorical_mappings = {
    'frequency': {
        'columns': ['FAM_DIF', 'DIFF_CONC_WORK', 'HH_TIRED', 'WORK_TIRED'],
        'order': FREQUENCY_ORDER
    },
    'task_division': {
        'columns': ['DIV_HH_COOK', 'DIV_HH_CLEAN', 'DIV_HH_GROC', 'DIV_HH_CARE', 'DIV_HH_LAUND', 'HH_WEEKEND'],
        'order': TASK_DIV_ORDER
    },
    'fairness_share': {
        'columns': ['SHARE_HH'],
        'order': FAIRNESS_SHARE_ORDER
    },
    'happiness': {
        'columns': ['LIFE_HAP'],
        'order': HAPPINESS_ORDER
    }
}

for cat_type, info in categorical_mappings.items():
    for col in info['columns']:
        for df in [df_2002_clean, df_2012_clean, df_2022_clean]:
            if col in df.columns:
                df[col] = pd.Categorical(df[col], categories=info['order'], ordered=True)

print(f"✓ Converted {sum(len(v['columns']) for v in categorical_mappings.values())} variables to ordered categorical")

# Display sample of cleaned data
print("\n" + "="*80)
print("SAMPLE OF CLEANED DATA")
print("="*80)

sample_vars = ['TOPBOT', 'SPWRKHRS', 'LIVWOMAR', 'WWYKS', 'SP_DEGREE', 'FAM_DIF', 'SHARE_HH', 'DIV_HH_COOK', 'LIFE_HAP', 'COHAB']
for var in sample_vars:
    if var in df_2022_clean.columns:
        print(f"\n{var}:")
        print(f"  2002: {df_2002_clean[var].value_counts().head(3).to_dict() if var in df_2002_clean.columns else 'N/A'}")
        print(f"  2012: {df_2012_clean[var].value_counts().head(3).to_dict() if var in df_2012_clean.columns else 'N/A'}")
        print(f"  2022: {df_2022_clean[var].value_counts().head(3).to_dict()}")

✓ Applied cleaning to 34 variables across all years
✓ Converted 12 variables to ordered categorical

SAMPLE OF CLEANED DATA

TOPBOT:
  2002: {5.0: 6270, 6.0: 6125, 7.0: 4077}
  2012: {5.0: 14677, 6.0: 10766, 7.0: 8080}
  2022: {5.0: 10932, 6.0: 8891, 7.0: 7294}

SPWRKHRS:
  2002: {40.0: 5034, 50.0: 1285, 48.0: 1087}
  2012: {40.0: 6698, 50.0: 1735, 45.0: 1450}
  2022: {40.0: 5852, 50.0: 1358, 45.0: 1050}

LIVWOMAR:
  2002: {'Agree': 17856, 'Strongly agree': 11321, 'Disagree': 7154}
  2012: {'Agree': 19334, 'Strongly agree': 12071, 'Disagree': 11740}
  2022: {'Strongly agree': 15414, 'Agree': 14343, 'Neither agree nor disagree': 6579}

WWYKS:
  2002: {'Work part-time': 23770, 'Work full-time': 12451, 'Stay at home': 6377}
  2012: {'Work part-time': 24981, 'Work full-time': 24783, 'Stay at home': 5960}
  2022: {'Work full-time': 21289, 'Work part-time': 16511, 'Stay at home': 2766}

SP_DEGREE:
  2002: {'Upper secondary': 9193, 'Tertiary (Bachelor level)': 5139, 'Primary': 4372}
  2012: {

## Save Cleaned Data

All variables have been cleaned and standardized across years. Now saving to CSV files.

In [335]:
# Save cleaned dataframes to CSV
df_2002_clean.to_csv("../clean_csv/2002_clean.csv", index=False)
df_2012_clean.to_csv("../clean_csv/2012_clean.csv", index=False)
df_2022_clean.to_csv("../clean_csv/2022_clean.csv", index=False)

print("✓ Saved all cleaned dataframes to ../clean_csv/")
print(f"\n2002: {df_2002_clean.shape}")
print(f"2012: {df_2012_clean.shape}")
print(f"2022: {df_2022_clean.shape}")

✓ Saved all cleaned dataframes to ../clean_csv/

2002: (46638, 69)
2012: (61754, 69)
2022: (45762, 69)


## Verify Data Consistency Across Years

Check that cleaned variables have consistent values across all three years.

In [336]:
# Verification: Check consistency of cleaned variables across years
print("="*80)
print("VERIFICATION OF DATA CONSISTENCY")
print("="*80)

# Variables to verify (sample from each type)
verify_vars = [
    'TOPBOT', 'SPWRKHRS', 'SP_HH_FAM', 'SP_HH', 'HH_FAM',
    'WWYKS', 'WWYKUS', 'SP_DEGREE', 'MOMORFAF',
    'HHTODD', 'HHCHILDR', 'HHADULT', 'HOMPOP',
    'LIVWOMAR', 'MEWH', 'HW_FULFIL', 'WO_WANT', 
    'WW_FAM_SUFFER', 'WW_CHILD_SUFFER', 'WW_WARM',
    'FAM_DIF', 'DIFF_CONC_WORK', 'HH_TIRED', 'WORK_TIRED',
    'SHARE_HH', 'DIV_HH_COOK', 'DIV_HH_CLEAN', 'DIV_HH_GROC',
    'DIV_HH_CARE', 'DIV_HH_LAUND', 'HH_WEEKEND',
    'LIFE_HAP', 'COHAB', 'C_ALPHAN'
]

dfs_dict = {
    2002: df_2002_clean,
    2012: df_2012_clean,
    2022: df_2022_clean
}

consistency_report = []

for var in verify_vars:
    # Check if variable exists in at least one df
    exists_in = [year for year, df in dfs_dict.items() if var in df.columns]
    
    if not exists_in:
        continue
    
    print(f"\n{'='*80}")
    print(f"{var}")
    print(f"{'='*80}")
    print(f"Available in years: {exists_in}")
    
    # Get unique values from each year
    for year in [2002, 2012, 2022]:
        df = dfs_dict[year]
        if var in df.columns:
            unique_vals = df[var].dropna().unique()
            
            # For categorical, show categories
            if hasattr(df[var], 'cat'):
                print(f"\n{year} (categorical): {df[var].cat.categories.tolist()}")
                print(f"  Non-null count: {df[var].notna().sum()}")
            else:
                # Show sample of unique values
                if len(unique_vals) <= 10:
                    print(f"\n{year}: {sorted(unique_vals)}")
                else:
                    print(f"\n{year}: {len(unique_vals)} unique values")
                    print(f"  Sample: {sorted(unique_vals)[:5]}")
                print(f"  Non-null count: {df[var].notna().sum()}")
        else:
            print(f"\n{year}: Not available")
    
    # Check consistency
    all_values = []
    for year, df in dfs_dict.items():
        if var in df.columns:
            if hasattr(df[var], 'cat'):
                all_values.extend(df[var].cat.categories.tolist())
            else:
                all_values.extend(df[var].dropna().unique().tolist())
    
    unique_across_years = len(set(str(v) for v in all_values))
    
    # Determine if consistent
    is_consistent = True
    if len(exists_in) > 1:
        # Compare value types across years
        value_sets = []
        for year, df in dfs_dict.items():
            if var in df.columns:
                if hasattr(df[var], 'cat'):
                    value_sets.append(set(df[var].cat.categories.tolist()))
                else:
                    vals = df[var].dropna().unique()
                    value_sets.append(set(str(v) for v in vals))
        
        if len(value_sets) > 1:
            # Check if there's significant overlap or if they're all numeric
            if all(df[var].dtype in ['int64', 'float64'] for year, df in dfs_dict.items() if var in df.columns):
                is_consistent = True  # Numeric variables are consistent
            elif len(value_sets[0]) > 0:
                # For categorical, check if categories are similar
                all_cats = set()
                for vs in value_sets:
                    all_cats.update(vs)
                # If all years share same categories, consistent
                is_consistent = all(vs == value_sets[0] for vs in value_sets)
    
    status = "✓ CONSISTENT" if is_consistent else "⚠ CHECK NEEDED"
    print(f"\nStatus: {status}")
    consistency_report.append((var, status, exists_in))

print("\n" + "="*80)
print("SUMMARY")
print("="*80)
consistent_count = sum(1 for _, status, _ in consistency_report if "CONSISTENT" in status)
print(f"Total variables checked: {len(consistency_report)}")
print(f"Consistent: {consistent_count}")
print(f"Need review: {len(consistency_report) - consistent_count}")

VERIFICATION OF DATA CONSISTENCY

TOPBOT
Available in years: [2002, 2012, 2022]

2002: [np.float64(1.0), np.float64(2.0), np.float64(3.0), np.float64(4.0), np.float64(5.0), np.float64(6.0), np.float64(7.0), np.float64(8.0), np.float64(9.0), np.float64(10.0)]
  Non-null count: 26626

2012: [np.float64(1.0), np.float64(2.0), np.float64(3.0), np.float64(4.0), np.float64(5.0), np.float64(6.0), np.float64(7.0), np.float64(8.0), np.float64(9.0), np.float64(10.0)]
  Non-null count: 56907

2022: [np.float64(1.0), np.float64(2.0), np.float64(3.0), np.float64(4.0), np.float64(5.0), np.float64(6.0), np.float64(7.0), np.float64(8.0), np.float64(9.0), np.float64(10.0)]
  Non-null count: 42193

Status: ✓ CONSISTENT

SPWRKHRS
Available in years: [2002, 2012, 2022]

2002: 88 unique values
  Sample: [np.float64(1.0), np.float64(2.0), np.float64(3.0), np.float64(4.0), np.float64(5.0)]
  Non-null count: 17534

2012: 93 unique values
  Sample: [np.float64(1.0), np.float64(2.0), np.float64(3.0), np.float64

In [ ]:
cleaned_vars = [
    "urban_rural",
    "spouse_work_status",
    "sex",
    "code_higher_income",
    "code_income_control",
    "hh_wrk_hrs",
    "work_status",
    "marital",
    "wrk_hrs",
    "educ_4_label",
    "age",
]

In [4]:
df_2002_clean = pd.read_csv("../clean_csv/2002_clean.csv")
df_2012_clean = pd.read_csv("../clean_csv/2012_clean.csv")
df_2022_clean = pd.read_csv("../clean_csv/2022_clean.csv")

In [5]:
df_2002_clean["marital_new"] = df_2002["v202"]
df_2012_clean["marital_new"] = df_2012["MARITAL"]
df_2022_clean["marital_new"] = df_2022["MARITAL"]

In [6]:
def clean_marital_status(val, year):
    """
    Standardizes marital status categories across years.
    Returns: Married, Civil partnership, Widowed, Divorced, Separated, Single, or None
    """
    if pd.isna(val):
        return None
    
    val_str = str(val).lower()
    
    # Handle refused/no answer
    if 'refused' in val_str or 'no answer' in val_str or val_str.startswith('-'):
        return None
    
    # Civil partnership (separate from marriage)
    if 'civil partnership' in val_str and 'never' not in val_str and "separate" not in val_str:
        return "Civil partnership"
    
    # Married (but NOT civil partnership)
    if ('married' in val_str or 'marr' in val_str) and 'civil partnership' not in val_str:
        if 'never' not in val_str and 'separated' not in val_str and 'divorced' not in val_str:
            return "Married"
    
    # Widowed
    if 'widow' in val_str or 'died' in val_str:
        return "Widowed"
    
    # Divorced
    if 'divorced' in val_str or 'legally separated' in val_str:
        return "Divorced"
    
    # Separated (but still legally married/in partnership)
    if 'separated' in val_str and 'divorced' not in val_str and 'legally separated' not in val_str:
        return "Separated"
    
    # Single
    if 'single' in val_str or 'never' in val_str:
        return "Single"
    
    return None

# Apply to all three dataframes
df_2002_clean["marital_new"] = df_2002_clean["marital_new"].apply(lambda x: clean_marital_status(x, 2002))
df_2012_clean["marital_new"] = df_2012_clean["marital_new"].apply(lambda x: clean_marital_status(x, 2012))
df_2022_clean["marital_new"] = df_2022_clean["marital_new"].apply(lambda x: clean_marital_status(x, 2022))

# Check the distribution
print("2002 Marital Status Distribution:")
print(df_2002_clean["marital_new"].value_counts())
print("\n2012 Marital Status Distribution:")
print(df_2012_clean["marital_new"].value_counts())
print("\n2022 Marital Status Distribution:")
print(df_2022_clean["marital_new"].value_counts())

2002 Marital Status Distribution:
marital_new
Married      27103
Single       11396
Widowed       3781
Divorced      2944
Separated     1033
Name: count, dtype: int64

2012 Marital Status Distribution:
marital_new
Married              34308
Single               15081
Widowed               5158
Divorced              4197
Civil partnership     1265
Separated             1132
Name: count, dtype: int64

2022 Marital Status Distribution:
marital_new
Married              22862
Single               12857
Divorced              3726
Widowed               3334
Civil partnership     1221
Separated              887
Name: count, dtype: int64


In [7]:
df_2002_clean["marital"] = df_2002_clean["marital_new"]
df_2012_clean["marital"] = df_2012_clean["marital_new"]
df_2022_clean["marital"] = df_2022_clean["marital_new"]

In [8]:
df_2002_clean.drop("marital_new",axis=1, inplace=True)
df_2012_clean.drop("marital_new",axis=1, inplace=True)
df_2022_clean.drop("marital_new",axis=1, inplace=True)

In [11]:
[col for col in df_2022_clean.columns if col.startswith("marital")]

['marital']

In [12]:
# Save cleaned dataframes to CSV
df_2002_clean.to_csv("../clean_csv/2002_clean.csv", index=False)
df_2012_clean.to_csv("../clean_csv/2012_clean.csv", index=False)
df_2022_clean.to_csv("../clean_csv/2022_clean.csv", index=False)

print("✓ Saved all cleaned dataframes to ../clean_csv/")
print(f"\n2002: {df_2002_clean.shape}")
print(f"2012: {df_2012_clean.shape}")
print(f"2022: {df_2022_clean.shape}")

✓ Saved all cleaned dataframes to ../clean_csv/

2002: (46638, 69)
2012: (61754, 69)
2022: (45762, 69)
